In [14]:
import pandas as pd
import sklearn as sk
import numpy as np
from rdkit import Chem
from pathlib import Path

In [15]:
import glob
test_csvs = glob.glob("../data/test_*.csv")
tests = {Path(t).stem : pd.read_csv(t) for t in test_csvs}
tests.keys()

dict_keys(['test_mord3d', 'test_morgan', 'test_rdk', 'test_descriptors'])

In [16]:
train_csvs = glob.glob("../data/train_*.csv")
train = {Path(t).stem : pd.read_csv(t) for t in train_csvs}
train.keys()

dict_keys(['train_mord3d', 'train_centroid_distances', 'train_morgan', 'train_descriptors', 'train_crystals', 'train_distances', 'train_rdk'])

In [18]:
train['train_crystals'][]

Index(['identifiers', 'n_heavy_atoms', 'smiles', 'a', 'b', 'c', 'alpha',
       'beta', 'gamma', 'z_value', 'z_prime', 'spacegroup_symbol',
       'cell_volume', 'calculated_density', 'packing_coefficient',
       'void_volume', 'is_centrosymmetric', 'is_sohncke', 'vdw_inter',
       'vdw_intra', 'hbond_inter', 'hbond_intra'],
      dtype='object')